In [ ]:
# Тема: Да се најдат податоците за цената на акциите и ESG score на компанијата Addidas за изминатите 4 години.
# Потоа, да се проучат податоците со користење на tsfresh.
# Да се направи benchmark на најмалку 3 модели за предикција на ESG score на компанијата имајќи ги предвид горенаведените податоци.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tensorflow tsfresh

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.5/176.5 kB 10.0 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.14.1 which is incompatible.


In [3]:
import pandas as pd
from tsfresh.feature_extraction import extract_features
from tsfresh.feature_extraction.settings import MinimalFCParameters
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import tensorflow as tf


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [53]:
# Преземање на податоците за акциите на Adidas (симбол: ADS.DE за германската берза)
# stock_data = yf.download('ADS.DE', start='2020-01-01', end='2024-01-01')


df_adidas_stock = pd.read_excel('/content/drive/MyDrive/adidas_stock_data (1).xlsx')
df_esg_score = pd.read_excel('/content/drive/MyDrive/esg_score.xlsx')

In [54]:
df_adidas_stock

,Year,Open,High,Low,Close,Adj Close,Volume
0,2020-01-02,289.200012,293.250000,289.200012,291.549988,281.179169,429314
1,2020-01-03,290.000000,292.250000,288.350006,291.950012,281.564941,440067
2,2020-01-06,288.950012,290.649994,285.149994,289.649994,279.346710,494435
3,2020-01-07,290.549988,296.500000,290.500000,294.750000,284.265350,489629
4,2020-01-08,294.000000,302.850006,293.500000,302.850006,292.077240,686646
...,...,...,...,...,...,...,...
1016,2023-12-21,195.039993,197.100006,193.259995,194.419998,193.829056,386794
1017,2023-12-22,183.000000,184.679993,181.679993,184.139999,183.580292,1035464
1018,2023-12-27,185.419998,186.520004,184.820007,185.559998,184.995987,337533
1019,2023-12-28,185.679993,185.919998,182.860001,184.100006,183.540421,323489


In [55]:
df_esg_score

,Date,ESG score
0,2020,79
1,2021,85
2,2022,89
3,2023,52


In [56]:
df_adidas_stock.isnull().sum()

,0
Year,0
Open,0
High,0
Low,0
Close,0
Adj Close,0
Volume,0


In [57]:
df_adidas_stock.to_csv('/content/drive/MyDrive/adidas_stock_data.csv', index=False)
df_esg_score.to_csv('/content/drive/MyDrive/esg_score.csv', index=False)

In [80]:
df_adidas_stock = pd.read_csv('/content/drive/MyDrive/adidas_stock_data.csv')
df_esg_score = pd.read_csv('/content/drive/MyDrive/esg_score.csv')

In [81]:
df_adidas_stock['Year'] = pd.to_datetime(df_adidas_stock['Year']).dt.year
df_combined = pd.merge(df_adidas_stock, df_esg_score, how='left', left_on='Year', right_on='Date')

In [82]:
df_combined.isnull().sum()

,0
Year,0
Open,0
High,0
Low,0
Close,0
Adj Close,0
Volume,0
Date,0
ESG score,0


In [83]:
df_combined.head(10)

,Year,Open,High,Low,Close,Adj Close,Volume,Date,ESG score
0,2020,289.200012,293.250000,289.200012,291.549988,281.179169,429314,2020,79
1,2020,290.000000,292.250000,288.350006,291.950012,281.564941,440067,2020,79
2,2020,288.950012,290.649994,285.149994,289.649994,279.346710,494435,2020,79
3,2020,290.549988,296.500000,290.500000,294.750000,284.265350,489629,2020,79
4,2020,294.000000,302.850006,293.500000,302.850006,292.077240,686646,2020,79
5,2020,307.149994,310.549988,304.950012,310.000000,298.972900,734200,2020,79
6,2020,311.450012,311.850006,307.049988,309.049988,298.056671,496127,2020,79
7,2020,310.350006,311.850006,308.399994,309.149994,298.153076,465002,2020,79
8,2020,309.700012,315.399994,308.200012,313.250000,302.107269,600907,2020,79
9,2020,313.600006,316.049988,311.950012,316.049988,304.807648,503382,2020,79


In [62]:
df_combined.tail(10)

,Year,Open,High,Low,Close,Adj Close,Volume,Date,ESG score
1011,2023,198.380005,198.800003,194.860001,197.399994,196.799988,785397,2023,52
1012,2023,197.979996,198.600006,190.720001,192.520004,191.934830,1287219,2023,52
1013,2023,191.100006,192.240005,189.779999,192.240005,191.655685,341040,2023,52
1014,2023,192.000000,193.399994,190.839996,193.399994,192.812149,396217,2023,52
1015,2023,194.399994,196.360001,191.759995,195.940002,195.344437,484778,2023,52
1016,2023,195.039993,197.100006,193.259995,194.419998,193.829056,386794,2023,52
1017,2023,183.000000,184.679993,181.679993,184.139999,183.580292,1035464,2023,52
1018,2023,185.419998,186.520004,184.820007,185.559998,184.995987,337533,2023,52
1019,2023,185.679993,185.919998,182.860001,184.100006,183.540421,323489,2023,52
1020,2023,184.820007,185.360001,184.160004,184.160004,183.600235,161239,2023,52


In [63]:
extracted_features = extract_features(df_combined.drop(columns=['ESG score']), column_id='Year', default_fc_parameters=MinimalFCParameters())

Feature Extraction: 100%|██████████| 28/28 [00:00<00:00, 2340.11it/s]


In [64]:
df_features = extracted_features.merge(df_esg_score, left_index=True, right_on='Date')

In [65]:
X = df_features.drop(columns=['ESG score', 'Date'])
y = df_features['ESG score']

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# scaler.fit(X_train)
# X_train = scaler.fit_transform(X_train)
# X_test= scaler.transform(X_test)

In [67]:
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [68]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
mse_rf = mean_squared_error(y_test, y_pred_rf)
print('Random Forest MSE:', mse_rf)

Random Forest MSE: 212.8681000000001


In [84]:
from sklearn.svm import SVR
svr_model = SVR(kernel='rbf')
svr_model.fit(X_train, y_train)
y_pred_svr = svr_model.predict(X_test)
mse_svr = mean_squared_error(y_test, y_pred_svr)
print('SVR MSE:', mse_svr)

SVR MSE: 36.39657029359096


In [85]:
from keras.models import Sequential
from keras.layers import Dense
import keras

In [86]:
scaler.fit(y_train.values.reshape(-1, 1))
Y_train = scaler.transform(y_train.values.reshape(-1, 1))
Y_test = scaler.transform(y_test.values.reshape(-1, 1))


In [87]:
model = Sequential()
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(optimizer='adam', loss='mean_squared_error')

In [88]:
model.fit(X_train, y_train, epochs=10, batch_size=8)


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 5552.3931
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 5541.4321
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 5530.6431
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 5519.4805
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 5508.0176
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 5496.1714
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 5484.2583
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 5473.0562
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 5462.0039
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 5450.7334


In [89]:
preds = model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step


In [90]:
preds = scaler.inverse_transform(preds)
Y_test = scaler.inverse_transform(Y_test)

In [91]:
mean_squared_error(Y_test, preds), mean_absolute_error(Y_test, preds)

(2719.6202719120774, 52.14997863769531)

In [92]:
def evaluate_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    print(f'MSE: {mse}')
    print(f'MAE: {mae}')


print("Random Forest Evaluation:")
evaluate_model(y_test, y_pred_rf)

print("SVR Evaluation:")
evaluate_model(y_test, y_pred_svr)

print("ANN Evaluation:")
evaluate_model(y_test, preds)

Random Forest Evaluation:
MSE: 212.8681000000001
MAE: 14.590000000000003
SVR Evaluation:
MSE: 36.39657029359096
MAE: 6.032957010752767
ANN Evaluation:
MSE: 2719.6202719120774
MAE: 52.14997863769531
